<a href="https://colab.research.google.com/github/minshyee/CP1/blob/main/Experiment_model_keyBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topic Modeling


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/AIBootCamp/cp1/Suicide_Detection.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
origin = df.copy()

In [ ]:
df

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide
...,...,...
232069,If you don't like rock then your not going to ...,non-suicide
232070,You how you can tell i have so many friends an...,non-suicide
232071,pee probably tastes like salty tea😏💦‼️ can som...,non-suicide
232072,The usual stuff you find hereI'm not posting t...,suicide


In [ ]:
appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

In [ ]:
ecList_compiled = re.compile('|'.join(appos.keys()))

def expandContractions(text, ecList_compiled=ecList_compiled):
    def replace(match):
        return appos[match.group(0)]
    return ecList_compiled.sub(replace, text)

In [ ]:
expandContractions(df['text'][0])

"Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me twice and lied to me so much that I have decided to refuse to go back to her. As of a few days ago, she began threatening suicide. I have tirelessly spent these paat few days talking her out of it and she keeps hesitating because she wants to believe I'll come back. I know a lot of people will threaten this in order to get their way, but what happens if she really does? What do I do and how am I supposed to handle her death on my hands? I still love my wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm worried today may be the day she does it and I hope so much it does not happen."

## 1. KeyBERT

In [ ]:
!pip install sentence_transformers

In [ ]:
import numpy as np
import itertools

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

### 실물구현


In [ ]:
doc = df['text'][0]

In [ ]:
# 3개의 단어 묶음인 단어구 추출
n_gram_range = (3, 3)
stop_words = "english"

count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([df['text'][0]])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 54
trigram 다섯개만 출력 : ['ago began threatening' 'began threatening suicide' 'believe ll come'
 'cheated constantly feeling' 'cheated twice lied']


In [ ]:
# keyword 수치화
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([df['text'][0]])
candidate_embeddings = model.encode(candidates)

In [ ]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
keywords

['wife good cheated',
 'threatening suicide tirelessly',
 'ex wife threatening',
 'suiciderecently left wife',
 'wife threatening suiciderecently']

## KeyBERT Library

In [ ]:
!pip install keybert

In [ ]:
from keybert import KeyBERT

In [ ]:
kw_model = KeyBERT()

Basic

In [ ]:
keyword = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', top_n=5)
keyword

[('wife threatening suiciderecently', 0.652),
 ('suiciderecently left wife', 0.5853),
 ('ex wife threatening', 0.5285),
 ('threatening suiciderecently left', 0.5013),
 ('threatening suicide tirelessly', 0.4626)]

keyphrase_ngram_range=(1, 1)

In [ ]:
keyword = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=5, highlight=True)

Ex Wife Threatening SuicideRecently I left my wife for good because she has cheated on me 
twice and lied to me so much that I have decided to refuse to go back to her. As of a few 
days ago, she began threatening suicide. I have tirelessly spent these paat few days talking 
her out of it and she keeps hesitating because she wants to believe I'll come back. I know a 
lot of people will threaten this in order to get their way, but what happens if she really 
does? What do I do and how am I supposed to handle her death on my hands? I still love my 
wife but I cannot deal with getting cheated on again and constantly feeling insecure. I'm 
worried today may be the day she does it and I hope so much it doesn't happen.

Max Sum Similarity : 데이터 쌍 사이의 최대 합 거리는 데이터 쌍 간의 거리가 최대화되는 데이터 쌍으로 정의됩니다. 여기서의 의도는 후보 간의 유사성을 최소화하면서 문서와의 후보 유사성을 극대화하고자 하는 것입니다.

In [ ]:
keyword = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 3), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)
keyword

[('handle death hands', 0.0041),
 ('cheated twice lied', 0.1983),
 ('wife good cheated', 0.4441),
 ('suiciderecently left', 0.0849),
 ('ex wife threatening', 0.0815)]

Maximal Marginal Relevance : 결과를 다양화하는 마지막 방법은 MMR(Maximum Limit Relegance)입니다. MMR은 텍스트 요약 작업에서 중복을 최소화하고 결과의 다양성을 극대화하기 위해 노력합니다. 참고 할 수 있는 자료로 EmbedRank(https://arxiv.org/pdf/1801.04470.pdf) 라는 키워드 추출 알고리즘은 키워드/키프레이즈를 다양화하는 데 사용할 수 있는 MMR을 구현했습니다. 먼저 문서와 가장 유사한 키워드/키프레이즈를 선택합니다. 그런 다음 문서와 유사하고 이미 선택된 키워드/키프레이즈와 유사하지 않은 새로운 후보를 반복적으로 선택합니다.

In [ ]:
keyword = kw_model.extract_keywords(doc, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.7)
keyword

[('wife threatening suiciderecently', 0.652),
 ('cheated twice lied', 0.3034),
 ('does hope doesn', 0.1209),
 ('feeling insecure worried', 0.2322),
 ('order way happens', 0.0404)]

## 2. BERTopic

In [ ]:
!pip install bertopic[use]

     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 462 kB 43.4 MB/s 


In [ ]:
from bertopic import BERTopic

TypeError: ignored

In [ ]:
docs = df['text']

In [ ]:
model = BERTopic()
topics, probabilities = model.fit_transform(docs)

NameError: ignored

In [ ]:
print('각 문서의 토픽 번호 리스트 :',len(topics))
print('첫번째 문서의 토픽 번호 :', topics[0])